## 起始設定

In [ ]:
import talib
import numpy as np
import pandas as pd
from finlab.data import Data

folder = '/Volumes/Macintosh HD/history'

stock_id = '2412'

data = Data(folder)

## 取得預設持有報酬率

In [ ]:
open_adj = data.get_adj('開盤價')

profit1 =  open_adj.shift(-2) / open_adj.shift(-1)
profit5 =  open_adj.shift(-6) / open_adj.shift(-1)
profit10 = open_adj.shift(-11) / open_adj.shift(-1)
profit60 = open_adj.shift(-61) / open_adj.shift(-1)

profit1 = profit1.unstack()
profit5 = profit5.unstack()
profit10 = profit10.unstack()
profit60 = profit60.unstack()

## 分析candlestick

In [ ]:
from talib import abstract

strategies = {}

candlesticks = [fname for fname in dir(abstract) if fname[:3] == 'CDL']

for index, fname in enumerate(candlesticks, 1):
    print(f'{index}/{len(candlesticks)}: {fname} is running')

    df = data.talib(fname)
    df = df.unstack()
    df = df[df!= 0]

    stock_series = df.dropna(axis=0)

    if stock_series.empty:
        continue

    pos = stock_series > 0
    neg = stock_series < 0
    
    # profit 用cumprod 算出總報酬率
    # 用mean 可算出該蠟燭形態 的報酬率都落在什麼位置
    # mean 大於 1 至少可說這報酬率是正值 會賺錢
    strategies[fname] = {
        "positive_profit_1": profit1.loc[pos.index].unstack(level=0).mean(),
        "positive_profit_5": profit5.loc[pos.index].unstack(level=0).mean(),
        "positive_profit_10": profit10.loc[pos.index].unstack(level=0).mean(),
        "negative_profit_1": profit1.loc[neg.index].unstack(level=0).mean(),
        "negative_profit_5": profit5.loc[neg.index].unstack(level=0).mean(),
        "negative_profit_10": profit10.loc[neg.index].unstack(level=0).mean(),
    }

In [ ]:
for name, strategy in strategies.items():
    print(name)
    print('positive: ' + str(len(strategy['positive'])))
    print('negative: ' + str(len(strategy['negative'])))

## 將所有報酬率統整到dataframe
用 Multiple Index 的方式 將 candle stick name & stock id 作為 indies 
並且用各種profit當作 column 組合出一個dataframe

In [ ]:
df_ = []
candlesitck_names = []
for candlesitck in strategies:
    df = pd.DataFrame(strategies[candlesitck])
    candlesitck_names.append(candlesitck)
    df_.append(df)

df_total = pd.concat(df_, keys=candlesitck_names, axis=0)

# 將 mutiple index 依照類型命名
df_total.index.names = ['candlestick', dfy.index.names[1]]
df_total

df_total.plot()

In [ ]:
df_profit = pd.DataFrame(strategies['CDLBELTHOLD'])
for column in df_profit.columns:
    if not 'positive_profit' in column:
        continue
    ret = df_profit.shift(-1)[column] - df_profit[column]
    cret_total = ret.cumprod()
    ax1 = cret_total.plot(label=column.replace('_profit_','_'))
    h1, l1 = ax1.           get_legend_handles_labels()

import matplotlib.pyplot as plt

plt.legend(h1, l1, loc=2)

plt.show()

In [ ]:
strategies_ = {}
fname = 'CDL3INSIDE'
df = data.talib(fname)
df = df.unstack()
df = df[df!= 0]

stock_series = df.dropna(axis=0)

pos = stock_series > 0
neg = stock_series < 0

# profit 用cumprod 算出總報酬率
# 用mean 可算出該蠟燭形態 的報酬率都落在什麼位置
# mean 大於 1 至少可說這報酬率是正值 會賺錢
strategies_[fname] = {
    "positive_profit_1": profit1.loc[pos.index].unstack(level=0).mean(),
    "positive_profit_5": profit5.loc[pos.index].unstack(level=0).mean(),
    "positive_profit_10": profit10.loc[pos.index].unstack(level=0).mean(),
    "negative_profit_1": profit1.loc[neg.index].unstack(level=0).mean(),
    "negative_profit_5": profit5.loc[neg.index].unstack(level=0).mean(),
    "negative_profit_10": profit10.loc[neg.index].unstack(level=0).mean(),
}

In [ ]:
open_adj = data.get_adj('開盤價')
ret = open_adj.shift(-1)[stock_id] - open_adj[stock_id]

cret_total = ret.cumsum()

cret_total.plot()

In [ ]:
profit10.loc[pos.index]['0015']

In [ ]:
p = profit10.loc[pos.index]['0015'].cumprod()

p.plot()

In [ ]:
profit10.loc[pos.index]['0015'].mean()

In [ ]:
profit5.loc[pos.index].unstack(level=0).mean()

In [ ]:
df1 = pd.DataFrame(strategies['CDLSHOOTINGSTAR'])
df1.index.name = 'stock_id'
df2 = pd.DataFrame(strategies['CDL3INSIDE'])
df2.index.name = 'stock_id'

df3 = pd.DataFrame(strategies['CDL3OUTSIDE'])
df3.index.name = 'stock_id'

df4 = pd.DataFrame(strategies['CDLTAKURI'])
df4.index.name = 'stock_id'

dfx = [df1, df2, df3, df4]

dfy = pd.concat(dfx,keys= ['CDLSHOOTINGSTAR','CDL3INSIDE','CDL3OUTSIDE','CDLTAKURI'],axis=0)

dfy.index.names = ['candlestick',dfy.index.names[1]]

dfy

In [ ]:
df_ = []
candlesitck_names = []
for candlesitck in strategies:
    df = pd.DataFrame(strategies[candlesitck])
    candlesitck_names.append(candlesitck)
    df_.append(df)

dfy = pd.concat(df_, keys=candlesitck_names, axis=0)

dfy.index.names = ['candlestick', dfy.index.names[1]]
dfy